In [1]:
import pandas as pd

df = pd.read_csv("warehouse_sales_export.csv", )
df.rename(columns={"ds": "date", "y": "quantity"}, inplace=True)

print(df.head())


         date category      region  quantity
0  2023-01-01    dairy    Barishal     182.0
1  2023-01-01    dairy  Chattogram     198.0
2  2023-01-01    dairy       Dhaka     146.0
3  2023-01-01    dairy      Khulna     187.0
4  2023-01-01    dairy  Mymensingh     225.0


In [12]:
print("Dataset info ")
print(df.info())
print("Dataset describe ")
print(df.describe())

Dataset info 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78912 entries, 0 to 78911
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ds        78912 non-null  object 
 1   category  78912 non-null  object 
 2   region    78912 non-null  object 
 3   y         78912 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.4+ MB
None
Dataset describe 
                  y
count  78912.000000
mean     209.007984
std       42.942520
min       32.000000
25%      179.000000
50%      207.000000
75%      237.000000
max      411.000000


In [13]:
# Check for missing values and handle them if necessary:
df.isnull().sum()


ds          0
category    0
region      0
y           0
dtype: int64

Data Processing

In [14]:
df['order_date'] = pd.to_datetime(df['ds'])


Feature Engineering

In [15]:
from IPython.core.display_functions import display
# Extract time-based features
df['day_of_week'] = df['order_date'].dt.dayofweek
df['month'] = df['order_date'].dt.month
df['quarter'] = df['order_date'].dt.quarter

# Lag features
df['lag_1'] = df['y'].shift(1)
df['lag_7'] = df['y'].shift(7)
df['lag_30'] = df['y'].shift(30)

# Rolling features
df['rolling_7'] = df['y'].rolling(7).mean()
df['rolling_30'] = df['y'].rolling(30).mean()

# One-hot encode categorical features
df = pd.get_dummies(df, columns=['region', 'category'])
df.dropna(inplace=True)  # remove rows with NaN after lag/rolling

display(df)

,ds,y,order_date,day_of_week,month,quarter,lag_1,lag_7,lag_30,rolling_7,...,region_Sylhet,category_dairy,category_fish,category_fruits,category_meat,category_oil,category_rice,category_snacks,category_spices,category_vegetables
30,2023-01-01,204.0,2023-01-01,6,1,1,173.0,195.0,182.0,214.142857,...,False,False,False,False,True,False,False,False,False,False
31,2023-01-01,179.0,2023-01-01,6,1,1,204.0,266.0,198.0,201.714286,...,True,False,False,False,True,False,False,False,False,False
32,2023-01-01,244.0,2023-01-01,6,1,1,179.0,278.0,146.0,196.857143,...,False,False,False,False,False,True,False,False,False,False
33,2023-01-01,139.0,2023-01-01,6,1,1,244.0,135.0,187.0,197.428571,...,False,False,False,False,False,True,False,False,False,False
34,2023-01-01,158.0,2023-01-01,6,1,1,139.0,217.0,225.0,189.000000,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78907,2025-12-31,233.0,2025-12-31,2,12,4,174.0,304.0,181.0,210.857143,...,False,False,False,False,False,False,False,False,False,True
78908,2025-12-31,212.0,2025-12-31,2,12,4,233.0,196.0,220.0,213.142857,...,False,False,False,False,False,False,False,False,False,True
78909,2025-12-31,201.0,2025-12-31,2,12,4,212.0,314.0,157.0,197.000000,...,False,False,False,False,False,False,False,False,False,True
78910,2025-12-31,209.0,2025-12-31,2,12,4,201.0,139.0,217.0,207.000000,...,False,False,False,False,False,False,False,False,False,True


Split Train/Test

In [16]:
train_size = int(len(df) * 0.8)
train = df.iloc[:train_size]
test = df.iloc[train_size:]

Model Selection

We will compare 3 models for demand forecast:

* Prophet (Time-series model)

* XGBoost / LightGBM (ML regression)

* Simple baseline (like last week average)

## Prophet Model

In [20]:
# ! pip install Prophet

In [22]:
from prophet import Prophet

prophet_df = df.groupby('order_date')['y'].sum().reset_index()
prophet_df.rename(columns={'order_date': 'ds',}, inplace=True)

model_prophet = Prophet(daily_seasonality=True, yearly_seasonality=True)
model_prophet.fit(prophet_df.iloc[:train_size])

future = model_prophet.make_future_dataframe(periods=len(test))
forecast = model_prophet.predict(future)


03:44:45 - cmdstanpy - INFO - Chain [1] start processing
03:44:45 - cmdstanpy - INFO - Chain [1] done processing


In [23]:
# Evaluate
from sklearn.metrics import mean_absolute_error

mae_prophet = mean_absolute_error(forecast['yhat'][-len(test):], test['y'])
print("Prophet MAE:", mae_prophet)

Prophet MAE: 14640.171502582376


## XGBoost / LightGBM Regression

In [30]:
df.columns

Index(['ds', 'y', 'order_date', 'day_of_week', 'month', 'quarter', 'lag_1',
       'lag_7', 'lag_30', 'rolling_7', 'rolling_30', 'region_Barishal',
       'region_Chattogram', 'region_Dhaka', 'region_Khulna',
       'region_Mymensingh', 'region_Rajshahi', 'region_Rangpur',
       'region_Sylhet', 'category_dairy', 'category_fish', 'category_fruits',
       'category_meat', 'category_oil', 'category_rice', 'category_snacks',
       'category_spices', 'category_vegetables'],
      dtype='object')

In [28]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# 1️⃣ Define features
features = [c for c in df.columns if c not in ['order_date', 'y']]

# 2️⃣ Convert datetime columns to numeric features
for df_split in [train, test]:
    if 'order_date' in df_split.columns:
        df_split['order_date'] = pd.to_datetime(df_split['order_date'])
        df_split['day'] = df_split['order_date'].dt.day
        df_split['month'] = df_split['order_date'].dt.month
        df_split['year'] = df_split['order_date'].dt.year
        df_split['day_of_week'] = df_split['order_date'].dt.dayofweek
        df_split.drop('order_date', axis=1, inplace=True)

# 3️⃣ Make sure all features are numeric
X_train = train[features].apply(pd.to_numeric, errors='coerce').fillna(0)
y_train = train['y']
X_test = test[features].apply(pd.to_numeric, errors='coerce').fillna(0)
y_test = test['y']

# 4️⃣ Train XGBoost
xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05)
xgb_model.fit(X_train, y_train)

# 5️⃣ Predict & Evaluate
y_pred_xgb = xgb_model.predict(X_test)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print("XGBoost MAE:", mae_xgb)

XGBoost MAE: 31.586419617577395


## Baseline Model

Simple last week average forecast:

In [31]:
y_pred_baseline = test['lag_7'].values
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
print("Baseline MAE:", mae_baseline)

Baseline MAE: 48.533815047220635


## Compare Models

In [32]:
print(f"MAE Comparison:")
print(f"Prophet: {mae_prophet:.2f}")
print(f"XGBoost: {mae_xgb:.2f}")
print(f"Baseline: {mae_baseline:.2f}")

MAE Comparison:
Prophet: 14640.17
XGBoost: 31.59
Baseline: 48.53


In [35]:
import pandas as pd

if mae_xgb < mae_prophet and mae_xgb < mae_baseline:
    final_model = xgb_model
    
    # Prepare next 30 rows for prediction
    future_features = df[features].tail(30).copy()

    # Convert datetime if present
    if 'order_date' in future_features.columns:
        future_features['order_date'] = pd.to_datetime(future_features['order_date'])
        future_features['day'] = future_features['order_date'].dt.day
        future_features['month'] = future_features['order_date'].dt.month
        future_features['year'] = future_features['order_date'].dt.year
        future_features['day_of_week'] = future_features['order_date'].dt.dayofweek
        future_features = future_features.drop('order_date', axis=1)
    
    # Drop any leftover object columns (like 'ds' from Prophet)
    object_cols = future_features.select_dtypes(include='object').columns
    future_features = future_features.drop(columns=object_cols)
    
    # Ensure numeric
    future_features = future_features.apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # Predict
    forecast_final = final_model.predict(future_features)
    
elif mae_prophet < mae_baseline:
    final_model = model_prophet
    future = final_model.make_future_dataframe(periods=30)
    forecast_final = final_model.predict(future)['yhat'][-30:]
else:
    forecast_final = df['lag_7'].tail(30).values  # baseline

ValueError: feature_names mismatch: ['ds', 'day_of_week', 'month', 'quarter', 'lag_1', 'lag_7', 'lag_30', 'rolling_7', 'rolling_30', 'region_Barishal', 'region_Chattogram', 'region_Dhaka', 'region_Khulna', 'region_Mymensingh', 'region_Rajshahi', 'region_Rangpur', 'region_Sylhet', 'category_dairy', 'category_fish', 'category_fruits', 'category_meat', 'category_oil', 'category_rice', 'category_snacks', 'category_spices', 'category_vegetables'] ['day_of_week', 'month', 'quarter', 'lag_1', 'lag_7', 'lag_30', 'rolling_7', 'rolling_30', 'region_Barishal', 'region_Chattogram', 'region_Dhaka', 'region_Khulna', 'region_Mymensingh', 'region_Rajshahi', 'region_Rangpur', 'region_Sylhet', 'category_dairy', 'category_fish', 'category_fruits', 'category_meat', 'category_oil', 'category_rice', 'category_snacks', 'category_spices', 'category_vegetables']
expected ds in input data

In [3]:
# Demand Forecasting (Category × Region) with 5 models
# - Builds lag/rolling/calendar features
# - Time-based split (last 30 days validation)
# - Trains 5 models, evaluates MAE, picks best
# - Forecasts next 15 days (recursive)
#
# Required df columns: date, category, region, quantity

import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor

# -----------------------------
# 1) Prepare data
# -----------------------------
df = df.copy()

# handle prophet-like columns if present
if "ds" in df.columns:
    df = df.rename(columns={"ds": "date"})
if "y" in df.columns:
    df = df.rename(columns={"y": "quantity"})

required = {"date", "category", "region", "quantity"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}. Required: {required}")

df["date"] = pd.to_datetime(df["date"])
df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce").fillna(0.0)
df = df.sort_values(["category", "region", "date"]).reset_index(drop=True)

# -----------------------------
# 2) Feature engineering
# -----------------------------
def make_features(data: pd.DataFrame) -> pd.DataFrame:
    d = data.sort_values(["category", "region", "date"]).copy()

    # calendar
    d["dow"] = d["date"].dt.dayofweek
    d["month"] = d["date"].dt.month
    d["is_weekend"] = (d["dow"] >= 5).astype(int)

    # lag + rolling per category-region
    g = d.groupby(["category", "region"], sort=False)

    d["lag_1"] = g["quantity"].shift(1)
    d["lag_7"] = g["quantity"].shift(7)
    d["lag_14"] = g["quantity"].shift(14)
    d["rollmean_7"] = g["quantity"].shift(1).rolling(7).mean()

    return d

feat = make_features(df)

# remove early rows (need history)
feat = feat.dropna(subset=["lag_14", "rollmean_7"]).reset_index(drop=True)

# -----------------------------
# 3) Time-based split (last 30 days)
# -----------------------------
VAL_DAYS = 30
max_date = feat["date"].max()
val_start = max_date - pd.Timedelta(days=VAL_DAYS - 1)

train_df = feat[feat["date"] < val_start].copy()
val_df = feat[feat["date"] >= val_start].copy()

print("Train:", train_df["date"].min().date(), "->", train_df["date"].max().date(), "rows:", len(train_df))
print("Val:  ", val_df["date"].min().date(), "->", val_df["date"].max().date(), "rows:", len(val_df))

# -----------------------------
# 4) Prepare X/y
# -----------------------------
cat_cols = ["category", "region"]
num_cols = ["dow", "month", "is_weekend", "lag_1", "lag_7", "lag_14", "rollmean_7"]

X_train = train_df[cat_cols + num_cols]
y_train = train_df["quantity"].astype(float)

X_val = val_df[cat_cols + num_cols]
y_val = val_df["quantity"].astype(float)

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), num_cols),
    ]
)

# -----------------------------
# 5) Define 5 models
# -----------------------------
models = {
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "Lasso": Lasso(alpha=0.001, random_state=42, max_iter=5000),
    "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.5, random_state=42, max_iter=5000),
    "RandomForest": RandomForestRegressor(
        n_estimators=400, random_state=42, n_jobs=-1, min_samples_leaf=2
    ),
    "HistGB": HistGradientBoostingRegressor(
        random_state=42, max_depth=8, learning_rate=0.07, max_iter=400
    ),
}

# -----------------------------
# 6) Train + Evaluate
# -----------------------------
results = []
fitted = {}

for name, model in models.items():
    pipe = Pipeline([("prep", preprocess), ("model", model)])
    pipe.fit(X_train, y_train)

    pred = pipe.predict(X_val)
    pred = np.clip(pred, 0, None)  # demand can't be negative

    mae = mean_absolute_error(y_val, pred)
    results.append({"model": name, "val_MAE": mae})
    fitted[name] = pipe

results_df = pd.DataFrame(results).sort_values("val_MAE").reset_index(drop=True)
print("\nModel Evaluation (lower MAE is better):")
print(results_df)

best_model_name = results_df.loc[0, "model"]
best_model = fitted[best_model_name]
print("\nBest model selected:", best_model_name)

# -----------------------------
# 7) Retrain best model on ALL data
# -----------------------------
X_all = feat[cat_cols + num_cols]
y_all = feat["quantity"].astype(float)

best_model.fit(X_all, y_all)
print("Best model retrained on full dataset.")

# -----------------------------
# 8) Forecast next 15 days (recursive)
# -----------------------------
HORIZON = 15

def forecast_next_days(df_original: pd.DataFrame, trained_model, horizon: int = 15) -> pd.DataFrame:
    hist = df_original.copy()
    hist["date"] = pd.to_datetime(hist["date"])
    hist = hist.sort_values(["category", "region", "date"]).reset_index(drop=True)

    last_date = hist["date"].max()
    future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=horizon, freq="D")

    forecasts = []

    for (cat, reg), g in hist.groupby(["category", "region"], sort=False):
        g = g.sort_values("date").copy()

        for d in future_dates:
            # add placeholder row
            g = pd.concat([g, pd.DataFrame([{
                "date": d, "category": cat, "region": reg, "quantity": np.nan
            }])], ignore_index=True)

            # build features for this group (so lags are correct)
            gg = make_features(g)
            row = gg.iloc[-1]

            # if not enough history
            if pd.isna(row["lag_14"]) or pd.isna(row["rollmean_7"]):
                yhat = 0.0
            else:
                X_row = pd.DataFrame([{
                    "category": cat,
                    "region": reg,
                    "dow": int(row["dow"]),
                    "month": int(row["month"]),
                    "is_weekend": int(row["is_weekend"]),
                    "lag_1": float(row["lag_1"]),
                    "lag_7": float(row["lag_7"]),
                    "lag_14": float(row["lag_14"]),
                    "rollmean_7": float(row["rollmean_7"]),
                }])

                yhat = float(trained_model.predict(X_row)[0])
                yhat = max(0.0, yhat)

            # write prediction back for next day's lag features
            g.loc[g.index[-1], "quantity"] = yhat

            forecasts.append({
                "date": d,
                "category": cat,
                "region": reg,
                "predicted_quantity": yhat
            })

    return pd.DataFrame(forecasts)

forecast_15 = forecast_next_days(df, best_model, horizon=HORIZON)

print("\nForecast sample:")
print(forecast_15.head(10))

forecast_15.to_csv("forecast_15_days.csv", index=False)
print("\nSaved: forecast_15_days.csv")

Train: 2023-01-15 -> 2025-12-01 rows: 75744
Val:   2025-12-02 -> 2025-12-31 rows: 2160

Model Evaluation (lower MAE is better):
          model    val_MAE
0    ElasticNet  34.566351
1         Lasso  34.566803
2         Ridge  34.566905
3        HistGB  34.640254
4  RandomForest  35.098464

Best model selected: ElasticNet
Best model retrained on full dataset.

Forecast sample:
        date category    region  predicted_quantity
0 2026-01-01    dairy  Barishal          207.383576
1 2026-01-02    dairy  Barishal          208.191854
2 2026-01-03    dairy  Barishal          207.400219
3 2026-01-04    dairy  Barishal          207.241983
4 2026-01-05    dairy  Barishal          207.699367
5 2026-01-06    dairy  Barishal          207.665238
6 2026-01-07    dairy  Barishal          208.327115
7 2026-01-08    dairy  Barishal          207.913731
8 2026-01-09    dairy  Barishal          208.357284
9 2026-01-10    dairy  Barishal          207.248266

Saved: forecast_15_days.csv


In [4]:
import os
import json
import joblib
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor


# =============================
# CONFIG
# =============================
VAL_DAYS = 30
MODEL_DIR = "saved_model"
MODEL_PATH = os.path.join(MODEL_DIR, "best_model.pkl")
META_PATH = os.path.join(MODEL_DIR, "meta.json")


# =============================
# FEATURE ENGINEERING
# =============================
def make_features(data: pd.DataFrame) -> pd.DataFrame:
    d = data.sort_values(["category", "region", "date"]).copy()

    # calendar
    d["dow"] = d["date"].dt.dayofweek
    d["month"] = d["date"].dt.month
    d["is_weekend"] = (d["dow"] >= 5).astype(int)

    # lags/rolling by group
    g = d.groupby(["category", "region"], sort=False)

    d["lag_1"] = g["quantity"].shift(1)
    d["lag_7"] = g["quantity"].shift(7)
    d["lag_14"] = g["quantity"].shift(14)
    d["rollmean_7"] = g["quantity"].shift(1).rolling(7).mean()

    return d


def train_and_save(df: pd.DataFrame):
    # basic cleanup
    df = df.copy()
    if "ds" in df.columns:
        df = df.rename(columns={"ds": "date"})
    if "y" in df.columns:
        df = df.rename(columns={"y": "quantity"})

    df["date"] = pd.to_datetime(df["date"])
    df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce").fillna(0.0)
    df = df.sort_values(["category", "region", "date"]).reset_index(drop=True)

    feat = make_features(df)
    feat = feat.dropna(subset=["lag_14", "rollmean_7"]).reset_index(drop=True)

    # time split
    max_date = feat["date"].max()
    val_start = max_date - pd.Timedelta(days=VAL_DAYS - 1)

    train_df = feat[feat["date"] < val_start].copy()
    val_df   = feat[feat["date"] >= val_start].copy()

    cat_cols = ["category", "region"]
    num_cols = ["dow", "month", "is_weekend", "lag_1", "lag_7", "lag_14", "rollmean_7"]

    X_train = train_df[cat_cols + num_cols]
    y_train = train_df["quantity"].astype(float)

    X_val = val_df[cat_cols + num_cols]
    y_val = val_df["quantity"].astype(float)

    preprocess = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), num_cols),
        ]
    )

    # 5 models
    models = {
        "Ridge": Ridge(alpha=1.0, random_state=42),
        "Lasso": Lasso(alpha=0.001, random_state=42, max_iter=5000),
        "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.5, random_state=42, max_iter=5000),
        "RandomForest": RandomForestRegressor(
            n_estimators=400, random_state=42, n_jobs=-1, min_samples_leaf=2
        ),
        "HistGB": HistGradientBoostingRegressor(
            random_state=42, max_depth=8, learning_rate=0.07, max_iter=400
        ),
    }

    # train + evaluate
    results = []
    fitted = {}

    for name, model in models.items():
        pipe = Pipeline([("prep", preprocess), ("model", model)])
        pipe.fit(X_train, y_train)

        pred = np.clip(pipe.predict(X_val), 0, None)
        mae = mean_absolute_error(y_val, pred)

        results.append({"model": name, "val_MAE": float(mae)})
        fitted[name] = pipe

    results_df = pd.DataFrame(results).sort_values("val_MAE").reset_index(drop=True)
    best_name = results_df.loc[0, "model"]
    best_pipe = fitted[best_name]

    print("Model leaderboard:")
    print(results_df)
    print("Best model:", best_name)

    # retrain best on all data
    X_all = feat[cat_cols + num_cols]
    y_all = feat["quantity"].astype(float)
    best_pipe.fit(X_all, y_all)

    # save
    os.makedirs(MODEL_DIR, exist_ok=True)
    joblib.dump(best_pipe, MODEL_PATH)

    meta = {
        "best_model": best_name,
        "val_days": VAL_DAYS,
        "feature_columns": {"cat": cat_cols, "num": num_cols},
        "leaderboard": results_df.to_dict(orient="records"),
        "trained_until": str(df["date"].max().date()),
    }
    with open(META_PATH, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

    print(f"Saved model -> {MODEL_PATH}")
    print(f"Saved meta  -> {META_PATH}")


# =============================
# RUN (example)
# =============================
if __name__ == "__main__":
  
    train_and_save(df)  # assumes df already exists

Model leaderboard:
          model    val_MAE
0    ElasticNet  34.566351
1         Lasso  34.566803
2         Ridge  34.566905
3        HistGB  34.640254
4  RandomForest  35.098464
Best model: ElasticNet
Saved model -> saved_model\best_model.pkl
Saved meta  -> saved_model\meta.json
